In [ ]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 7.5 MB/s 
     |████████████████████████████████| 9.3 MB 10.0 MB/s 
     |████████████████████████████████| 1.6 MB 43.5 MB/s 
     |████████████████████████████████| 108 kB 49.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [ ]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [ ]:
df.drop(['packer_type'],axis=1,inplace=True)

In [ ]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,1,3,0,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,216,5,1,1,1,...,1,0,5,0,6.174602,3.155928,365568,7.546568,0,1
5206,80,2,4,65535,184,256,7,1,0,1,...,1,4,3,0,0.000000,0.000000,98816,6.947195,0,1
5207,144,3,4,65535,184,216,5,1,0,1,...,1,2,3,0,6.503422,3.790871,227328,7.823114,0,1
5208,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,6.115208,7.919091,271616,7.886012,0,1


In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
658,144,3,4,65535,184,256,5,1,0,1,...,1,1,4,0,6.450820,6.115667,299552,6.523706,1,0
1767,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,1,6.126842,0.628362,2211840,5.734108,1,0
4001,144,3,4,65535,184,208,3,1,1,1,...,1,2,1,1,0.000000,0.000000,62976,7.788860,0,1
4437,144,3,4,65535,184,216,3,1,0,1,...,1,0,3,0,7.284032,7.944639,251904,7.529611,0,1
5100,144,3,4,65535,184,192,5,1,1,1,...,1,3,2,1,7.976910,0.000000,97280,7.668947,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000,144,3,4,65535,184,304,6,1,0,1,...,1,2,4,0,6.451145,4.943512,213648,6.580918,1,0
3141,144,3,4,65535,184,248,4,1,1,1,...,1,0,4,0,6.762466,7.951370,130560,6.934798,0,1
4238,144,3,4,65535,184,224,3,1,1,1,...,1,2,1,1,0.000000,0.000000,240640,7.976064,1,1
1423,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.140359,2.863678,56320,6.103309,1,0


In [ ]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [ ]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [ ]:
df

In [ ]:
df.drop(['index'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
df_5percent=df[0:781] 


In [ ]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,5,1,0,1,...,1,1,4,0,6.450820,6.115667,299552,6.523706,1,0
1,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,1,6.126842,0.628362,2211840,5.734108,1,0
2,144,3,4,65535,184,208,3,1,1,1,...,1,2,1,1,0.000000,0.000000,62976,7.788860,0,1
3,144,3,4,65535,184,216,3,1,0,1,...,1,0,3,0,7.284032,7.944639,251904,7.529611,0,1
4,144,3,4,65535,184,192,5,1,1,1,...,1,3,2,1,7.976910,0.000000,97280,7.668947,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,80,2,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,29696,7.681355,0,1
777,144,3,4,65535,184,248,4,1,0,1,...,1,1,3,0,6.418138,5.710792,9166336,6.391190,1,0
778,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.046779,0.000000,5632,3.906189,0,0
779,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.613434,7.874120,129024,7.461757,1,1


In [ ]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (368, 69)
class 1: (413, 69)


In [ ]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [ ]:
df_5percent.info

<bound method DataFrame.info of      e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0     144.0   3.0        4.0     65535.0  184.0     256.0               5.0   
1     144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
2     144.0   3.0        4.0     65535.0  184.0     208.0               3.0   
3     144.0   3.0        4.0     65535.0  184.0     216.0               3.0   
4     144.0   3.0        4.0     65535.0  184.0     192.0               5.0   
..      ...   ...        ...         ...    ...       ...               ...   
776    80.0   2.0        4.0     65535.0  184.0     256.0               3.0   
777   144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
778   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
779   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
780   144.0   3.0        4.0     65535.0  184.0     248.0               4.0   

     CreationYear  

In [ ]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,781.000000,781.000000,781.000000,781.000000,781.000000,781.000000,781.000000,781.000000,781.000000,781.0,...,781.0,781.000000,781.000000,781.000000,781.000000,781.000000,7.810000e+02,781.000000,781.000000,781.000000
mean,138.898848,2.909091,3.969270,65199.353393,267.111396,220.122919,4.581306,0.989757,0.343150,1.0,...,1.0,1.285531,3.295775,0.166453,4.962640,2.489017,8.714096e+05,6.326990,0.528809,0.528809
std,19.390621,0.355444,0.334477,4681.025323,2338.516951,51.562511,1.783862,0.100754,0.475065,0.0,...,0.0,1.436398,1.141679,0.372726,2.500737,2.675771,7.178608e+06,1.196328,0.499489,0.499489
min,0.000000,0.000000,0.000000,0.000000,0.000000,64.000000,1.000000,0.000000,0.000000,1.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942085,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,200.000000,4.000000,1.000000,0.000000,1.0,...,1.0,1.000000,3.000000,0.000000,4.178047,0.000000,6.297600e+04,5.681644,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.0,1.000000,4.000000,0.000000,6.148720,1.518354,1.228800e+05,6.396898,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.0,2.000000,4.000000,0.000000,6.497976,4.676743,3.220480e+05,7.376649,1.000000,1.000000
max,144.000000,3.000000,4.000000,65535.000000,65534.000000,648.000000,20.000000,1.000000,1.000000,1.0,...,1.0,15.000000,7.000000,1.000000,7.976910,7.999607,1.657081e+08,7.999864,1.000000,1.000000


In [ ]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [ ]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [ ]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [ ]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      56.25
1.0      43.75
dtype: float64

In [ ]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
0.0      50.0
1.0      50.0
dtype: float64

In [ ]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [ ]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [ ]:
circuit(weights_init, data[0])

tensor(-0.10834314, requires_grad=True)

In [ ]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [ ]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [ ]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.23370485, requires_grad=False), tensor(0.57855339, requires_grad=False)], Y =  1
X = [tensor(-0.28407902, requires_grad=False), tensor(-0.214514, requires_grad=False)], Y = -1
X = [tensor(-0.1985389, requires_grad=False), tensor(-0.21345908, requires_grad=False)], Y = -1
X = [tensor(-0.20248733, requires_grad=False), tensor(-0.13892623, requires_grad=False)], Y = -1
X = [tensor(-0.20082692, requires_grad=False), tensor(-0.14401566, requires_grad=False)], Y =  1


In [ ]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [ ]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0284233 | Accuracy: 0.5625000 
New best
Iter:     2 | Cost: 1.0689571 | Accuracy: 0.5687500 
New best
Iter:     3 | Cost: 1.0186897 | Accuracy: 0.5812500 
Iter:     4 | Cost: 0.9623039 | Accuracy: 0.5812500 
Iter:     5 | Cost: 0.9886717 | Accuracy: 0.4625000 
Iter:     6 | Cost: 1.0511494 | Accuracy: 0.4375000 
Iter:     7 | Cost: 1.0720641 | Accuracy: 0.4375000 
Iter:     8 | Cost: 1.0152314 | Accuracy: 0.4812500 
New best
Iter:     9 | Cost: 0.9704803 | Accuracy: 0.6187500 
Iter:    10 | Cost: 0.9787045 | Accuracy: 0.5687500 
Iter:    11 | Cost: 1.0297590 | Accuracy: 0.5687500 
Iter:    12 | Cost: 1.0145599 | Accuracy: 0.5687500 
Iter:    13 | Cost: 0.9684483 | Accuracy: 0.5750000 
Iter:    14 | Cost: 0.9671399 | Accuracy: 0.5750000 
Iter:    15 | Cost: 0.9558264 | Accuracy: 0.5875000 
Iter:    16 | Cost: 0.9704500 | Accuracy: 0.5937500 
Iter:    17 | Cost: 0.9640554 | Accuracy: 0.5937500 
Iter:    18 | Cost: 0.9543067 | Accuracy: 0.4812500 
Iter:    1

In [ ]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [ ]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9432141044540113, Accuracy: 55.00000000000001%


In [ ]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,-1.0
2,1.0,-1.0
3,1.0,1.0
4,1.0,-1.0
5,1.0,-1.0
6,1.0,-1.0
7,1.0,1.0
8,1.0,-1.0
9,1.0,1.0


In [ ]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.30      0.60      0.40        10
         1.0       0.80      0.53      0.64        30

    accuracy                           0.55        40
   macro avg       0.55      0.57      0.52        40
weighted avg       0.68      0.55      0.58        40

0.8
0.5333333333333333
0.64
0.5666666666666667
